# Анализ A/B-теста

In [26]:
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu

**Загрузка данных**

In [27]:
file_path = '/mnt/HC_Volume_18315164/home-jupyter/jupyter-ev-varlamov/homework_tasks/Проект_1_Задание_2.csv'
data = pd.read_csv(file_path, delimiter=';')
data

,user_id,revenue,testgroup
0,1,0,b
1,2,0,a
2,3,0,a
3,4,0,b
4,5,0,b
...,...,...,...
404765,404766,0,a
404766,404767,0,b
404767,404768,231,a
404768,404769,0,a


**Предварительный анализ данных**

In [28]:
data.head(), data.info(), data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404770 entries, 0 to 404769
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   user_id    404770 non-null  int64 
 1   revenue    404770 non-null  int64 
 2   testgroup  404770 non-null  object
dtypes: int64(2), object(1)
memory usage: 9.3+ MB


(   user_id  revenue testgroup
 0        1        0         b
 1        2        0         a
 2        3        0         a
 3        4        0         b
 4        5        0         b,
 None,
              user_id        revenue
 count  404770.000000  404770.000000
 mean   202385.500000      26.083435
 std    116847.178567     681.652928
 min         1.000000       0.000000
 25%    101193.250000       0.000000
 50%    202385.500000       0.000000
 75%    303577.750000       0.000000
 max    404770.000000   37433.000000)

**Разделение данных на контрольную и тестовую группы**

In [29]:
control_group = data[data['testgroup'] == 'a']
test_group = data[data['testgroup'] == 'b']

**Общее количество пользователей в каждой группе**

In [30]:
total_users_control = len(control_group)
total_users_test = len(test_group)

**Общий доход в каждой группе**

In [31]:
total_revenue_control = control_group['revenue'].sum()
total_revenue_test = test_group['revenue'].sum()

**Количество платящих пользователей в каждой группе**

In [32]:
paying_users_control = control_group[control_group['revenue'] > 0]['user_id'].nunique()
paying_users_test = test_group[test_group['revenue'] > 0]['user_id'].nunique()

**Вычисление ARPU**

In [33]:
arpu_control = total_revenue_control / total_users_control
arpu_test = total_revenue_test / total_users_test

**Вычисление конверсии**

In [34]:
cr_control = paying_users_control / total_users_control
cr_test = paying_users_test / total_users_test

**Вывод результатов**

In [35]:
print(f"ARPU в контрольной группе: {arpu_control}")
print(f"ARPU в тестовой группе: {arpu_test}")
print(f"Конверсия в контрольной группе: {cr_control}")
print(f"Конверсия в тестовой группе: {cr_test}")

ARPU в контрольной группе: 25.413719736965806
ARPU в тестовой группе: 26.75128659327863
Конверсия в контрольной группе: 0.009539690157988749
Конверсия в тестовой группе: 0.008906235351586593


**Сравнение ARPU**

Для сравнения ARPU между двумя группами (контрольной и тестовой) можно использовать t-тест для двух выборок. ARPU является непрерывной метрикой, и t-тест подходит для сравнения средних значений двух независимых выборок, если данные примерно **нормальны** и дисперсии примерно равны. Проверим нормальность данных с помощью теста Шапиро-Уилка.

**Проверка нормальности для ARPU данных**

In [13]:
shapiro_control_arpu = shapiro(control_group['revenue'])
shapiro_test_arpu = shapiro(test_group['revenue'])

print(f"Shapiro-Wilk тест для ARPU контрольной группы: statistic={shapiro_control_arpu.statistic}, p-value={shapiro_control_arpu.pvalue}")
print(f"Shapiro-Wilk тест для ARPU тестовой группы: statistic={shapiro_test_arpu.statistic}, p-value={shapiro_test_arpu.pvalue}")

Shapiro-Wilk тест для ARPU контрольной группы: statistic=0.008876502513885498, p-value=0.0
Shapiro-Wilk тест для ARPU тестовой группы: statistic=0.06588172912597656, p-value=0.0


/opt/tljh/user/lib/python3.7/site-packages/scipy/stats/morestats.py:1760: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


Результаты теста Шапиро-Уилка показывают, что данные для ARPU не следуют нормальному распределению (p-value = 0.0). В этом случае использование непараметрических тестов, таких как тест Манна-Уитни, будет более уместным.

**Сравнение ARPU с использованием теста Манна-Уитни**

In [21]:
arpu_mannwhitney_result = mannwhitneyu(control_group['revenue'], test_group['revenue'], alternative='two-sided')

print(f"Тест Манна-Уитни для ARPU: statistic={arpu_mannwhitney_result.statistic}, p-value={arpu_mannwhitney_result.pvalue}")

Тест Манна-Уитни для ARPU: statistic=20491259376.0, p-value=0.06269701316074398


**Сравнение конверсии**

Конверсия является долей, и z-тест для пропорций подходит для сравнения двух пропорций из независимых выборок.

In [15]:
count = [paying_users_control, paying_users_test]
nobs = [total_users_control, total_users_test]

conversion_ztest_result = proportions_ztest(count, nobs)

print(f"Z-тест для конверсии: statistic={conversion_ztest_result[0]}, p-value={conversion_ztest_result[1]}")

Z-тест для конверсии: statistic=2.108028495889841, p-value=0.035028524642854865


**Основные результаты**:

1. **ARPU**: в тестовой группе выше, но различие не статистически значимо на уровне 0.05 (p-value = 0.0627).

2. **Конверсия**: в тестовой группе ниже, и это различие статистически значимо (p-value = 0.035).

**Вывод**:

Так как ARPU в тестовой группе увеличилась незначительно, а конверсия значительно уменьшилась, распространять тестовый набор акционных предложений на всех пользователей **не рекомендуется**.